In [2]:
!pip install numpy
!pip install opencv-python
!pip install tensorflow
!pip install tflearn
!pip install keras
!pip install spotipy --upgrade
!pip install play_mood_music

     |████████████████████████████████| 112kB 6.6MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=54d557e4dfe63ae438c5c9a03c1f00fe771ee19117b9fce41cb34228617f0adf
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn
ERROR: Could not find a version that satisfies the requirement play_mood_music (from versions: none)
ERROR: No matching distribution found for play_mood_music


In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
import os
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import spotipy
import random
from spotipy.oauth2 import SpotifyOAuth


In [4]:
import numpy as np
import cv2
import matplotlib as mpl
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
# from play_mood_music import setup, next_track
from keras.layers.pooling import MaxPooling2D
import os

In [45]:

def authenticate(scope='user-modify-playback-state,user-read-playback-state'):
    return spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, redirect_uri="https://localhost:8000"))



In [15]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
mpl.use('TkAgg')

# settings
RUNNING_AVERAGE_SAMPLES = 5
LONGTERM_ROLLING_AVERAGE_SAMPLES = 5


In [17]:
# Query Spotify to get a new track matching the given mood


def get_new_track_uri(spotify, mood, playlist_offset=None, track_offset=None, instrumental=False):
    # query for a playlist with a matching "mood"

    if playlist_offset is None:
        playlist_offset = random.randint(0, 10)
    if track_offset is None:
        track_offset = random.randint(0, 20)

    if instrumental:
        results = spotify.search(q=mood + ' instrumental', type='playlist', limit=1, offset=playlist_offset)
    else:
        results = spotify.search(q=mood, type='playlist', limit=1, offset=playlist_offset)
    if len(results) < 1:
        return get_new_track_uri(spotify, mood, playlist_offset=0, instrumental=instrumental)
    playlist_id = results['playlists']['items'][0]['id']

    # query the selected playlist for a random track
    results = spotify.playlist_items(playlist_id, market="CA", limit=1, offset=track_offset)
    if len(results) < 1:
        return get_new_track_uri(spotify, mood, playlist_offset=playlist_offset, track_offset=0)
    uri = results['items'][0]['track']['uri']

    return uri

In [18]:
# Verify that there is an active device
def has_active_device(spotify):
    devices = spotify.devices()
    for d in devices['devices']:
        if d['is_active']:
            return True
    return False


In [19]:
# Fade the volume up
def fade_in(spotify):
    for i in range(25):
        spotify.volume(50+(i*2))


In [20]:
# Fade the volume down
def fade_out(spotify):
    for i in range(25):
        spotify.volume(100-(i*2))


In [21]:
# Put a given track next in queue and switch from the current song to it
# optional fade into new song
def change_songs(spotify, track_uri):
    if not has_active_device(spotify):
        print("err: no active device")
        quit()
    spotify.add_to_queue(track_uri)
    fade_out(spotify)
    spotify.next_track()
    fade_in(spotify)

In [22]:
# Get a song matching the mood and start playing it
def next_track(spotify, mood, instrumental=False):
    track_uri = get_new_track_uri(spotify, mood, instrumental=instrumental)
    change_songs(spotify, track_uri)

In [23]:
# Authenticates and sets volume to 100
# return authentication object
def setup(scope=None):
    if scope is not None:
        spotify = authenticate(scope)
    else:
        spotify = authenticate()

    if not has_active_device(spotify):
        print("err: no active devices")
        quit()
    spotify.volume(100)
    return spotify

In [24]:
# Plot of accuracy and loss
def plot_model_history(model_history):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    # summarize history for accuracy
    axs[0].plot(range(1, len(model_history.history['accuracy']) + 1), model_history.history['accuracy'])
    axs[0].plot(range(1, len(model_history.history['val_accuracy']) + 1), model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_history.history['accuracy']) + 1),
                      len(model_history.history['accuracy']) / 10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1), len(model_history.history['loss']) / 10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot.png')
    plt.show()

In [27]:
# Define data generators
train_dir = 'data/train'
val_dir = 'data/test'

num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 50

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#    train_dir,
#    target_size=(48, 48),
#    batch_size=batch_size,
#    color_mode="grayscale",
#    class_mode='categorical')

# validation_generator = val_datagen.flow_from_directory(
#    val_dir,
#    target_size=(48, 48),
#    batch_size=batch_size,
#    color_mode="grayscale",
#    class_mode='categorical')


In [28]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


In [30]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [31]:
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [32]:
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [4]:
spotify = setup()  # setup Spotify integration
model.load_weights('model.h5')
cv2.ocl.setUseOpenCL(False)

In [34]:
# Dictionaries
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Neutral", 3: "Happy",
                4: "Neutral", 5: "Sad", 6: "Happy"}

action_dict = {0: "Calming down user", 1: "Skip to next song", 2: "No change", 3: "Upbeat Environment",
               4: "No change", 5: "Calming down user", 6: "Upbeat Environment"}

spotipy_dict = {0: "Rock", 1: "skip", 2: "", 3: "Upbeat",
                4: "", 5: "Piano", 6: "Upbeat"}


In [4]:
# Webcam Feed (LIVE)
cap = cv2.VideoCapture(0)
rolling_samples = []
freqs = np.zeros(6, dtype=int)
longterm_rolling_average = []
longterm_freqs = np.zeros(6, dtype=int)
prevmaxmood = None
maxmood = 4
sample = 0
while True:
  # Find haar cascade to draw bounding box around face and eyes
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))

        # take rolling average
        if len(rolling_samples) >= RUNNING_AVERAGE_SAMPLES:
            oldest = rolling_samples.pop(0)
            freqs[oldest] -= 1

        # map surprised to happy
        if maxindex == 6:
            maxindex = 3

        rolling_samples.append(maxindex)
        freqs[maxindex] += 1
        maxavg = np.max(freqs)
        maxavgindex = np.where(freqs == maxavg)[0][0]

        # Every 5 samples, add the avg max to a rolling average
        if sample % 10 == 0:
            if len(longterm_rolling_average) >= LONGTERM_ROLLING_AVERAGE_SAMPLES:
                oldest = longterm_rolling_average.pop(0)
                longterm_freqs[oldest] -= 1
            longterm_rolling_average.append(maxavgindex)
            longterm_freqs[maxavgindex] += 1
            maxmood = np.where(longterm_freqs == np.max(longterm_freqs))[0][0]

            if prevmaxmood is None or prevmaxmood != maxmood:
                if maxmood == 1:
                    maxmood = prevmaxmood
                elif maxmood in [2, 4]:
                    continue
                prevmaxmood = maxmood
                next_track(spotify, spotipy_dict[maxmood])

        sample += 1

        cv2.putText(frame, emotion_dict[maxavgindex], (x + 20, y - 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, action_dict[maxavgindex], (x + 20, y - 60), cv2.FONT_ITALIC, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, emotion_dict[maxmood], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show frame to user
    cv2.imshow("Frame: Pres 'q' to exit the program", frame)
    key = cv2.waitKey(1) & 0xFF

    # Quit program by pressing 'q'
    if key == ord("q"):
        spotify.pause_playback()
        break


In [4]:

cap.release()
cv2.destroyAllWindows()

In [4]:

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

model_info = model.fit(
    train_generator,
    steps_per_epoch=num_train // batch_size,
    epochs=num_epoch,
    validation_data=validation_generator,
    validation_steps=num_val // batch_size)

plot_model_history(model_info)
model.save_weights('model_testing.h5')
